In [ ]:
import pandas as pd

def constructGraph(repoName, parents_df):
    graph = {}

    repo_df = parents_df.loc[parents_df['Repo Name'] == repoName]

    all_children = set(repo_df['Child'].tolist())

    for child in all_children:
        temp_df = repo_df.loc[repo_df['Child'] == child]

        graph[child] = tuple(temp_df['Parent'].tolist())

    return graph



In [ ]:
def getRemovedComments():
    df = pd.read_csv("main_only_label.csv")

    df = df.loc[df['Comment-Removing Revision'] != "STILL_EXISTS"]

    df['Comment-Introducing Revision'] = df['Comment-Introducing Revision'].apply(removeURL)
    df['Comment-Removing Revision'] = df['Comment-Removing Revision'].apply(removeURL)

    return df[['Dataset ID', 'Repo Name','Comment-Introducing Revision','Comment-Removing Revision', 'Comment Text', 'Filename']]

def removeURL(aString):
    #print(aString)
    result = aString.split("commit/")

    return result[1]

print(getRemovedComments())

In [ ]:
def goodMatch(nodeData, lookupID, removedData):
    filtered = removedData[(removedData['Column3'] == lookupID) & (removedData['Column2'] == nodeData[1]) & (removedData['Column4'] == nodeData[2]) & (removedData['Column5'] == nodeData[3]) & (removedData['Column6'] == 'ADDED')]

    return not filtered.empty

In [ ]:

def BFS(graph, start, removedData):
    queue = [(start[0], 0)]
    visited = set()

    while queue:
        temp = queue.pop(0)
        
        if temp[0] not in visited:

            if goodMatch(start, temp[0], removedData):
                return temp

            if temp[0] in graph:
                for parent in graph[temp[0]]:
                    queue.append((parent, temp[1] + 1))

            visited.add(temp[0])

In [ ]:
def getRemovalData():
    return pd.read_csv('8-autogen_removed_sorted.csv')

In [ ]:
def findLength():

    removal_df = getRemovalData()

    parents_df = pd.read_csv("parents_data.csv")

    parents_df["Child"]= parents_df["Child"].map(str)
    parents_df["Parent"]= parents_df["Parent"].map(str)

    comment_df = getRemovedComments()

    my_output = []

    for num,i in enumerate(range(len(comment_df))):

        graph = constructGraph(comment_df.iloc[i,1], parents_df)

        start_node = (comment_df.iloc[i,3], comment_df.iloc[i,1], comment_df.iloc[i,5], comment_df.iloc[i,4])

        #print(start_node)

        result = BFS(graph, start_node, removal_df)
        my_output.append(result)
        print(num, comment_df.iloc[i,0], result)
    
    return my_output

my_output = findLength()

In [ ]:
#replace the removal IDs with the removal IDs found via parent traversal

import numpy as np

comments_df = getRemovedComments()

ref = {}

for num,i in enumerate(range(len(comments_df))):
    ref[comments_df.iloc[i,0]] = my_output[num]

print(ref)

label_df = pd.read_csv('main_only_label.csv')

label_df['Number of Commits to Remove'] = np.nan

for num, i in enumerate(range(len(comments_df))):
    if label_df.iloc[i, 1] in ref:
        if ref[label_df.iloc[i, 1]] == None:
            label_df.iloc[i, 5] = 'UNTRACKABLE'
            label_df.iloc[i, 12] = 'UNTRACKABLE'

        else:
            label_df.iloc[i, 5] = 'https://github.com/' + label_df.iloc[i, 2] + '/commit/' + ref[label_df.iloc[i, 1]][0]
            label_df.iloc[i, 12] = ref[label_df.iloc[i, 1]][1]
    else:
        label_df.iloc[i, 12] = 'NOT A REMOVAL'